In [17]:
from pycparser import parse_file, c_ast
from numpy import inf
from copy import copy

In [18]:
def parse_c_file(file):
    """
        Parse a c_file via pycparser
        
        Arguments
        ----------
        file: String
            c file to parse
    """
    ast = parse_file(file, use_cpp=True,
        cpp_path='gcc',
        cpp_args=['-E', r'-Iutils/fake_libc_include'])
    return ast

In [19]:
ast_graph = parse_c_file('./C files/if.c')
foo_graph = ast_graph.ext[0]

In [20]:
class Value():
    """
        Représente une valeur de variable qui est sous la forme: String + Constante
        
        Attributs
        ----------
        constante: Int
            Constante
        variable: Str
            name variable
    """
    def __init__(self, constante = 0, variable = None):
        self.variable = variable
        self.constante = constante
    
    def compute(ast_node):
        if type(ast_node) is c_ast.Constant:
            return Value(constante = int(ast_node.value))
        
        elif type(ast_node) is c_ast.ID:
            return Value(variable=ast_node.name)
        
        elif type(ast_node) is c_ast.BinaryOp:
            left, right, op = ast_node.left, ast_node.right, ast_node.op 
            
            if type(left) is c_ast.ID and type(right) is c_ast.ID:
                raise Exception("Operation of two variables")
            elif type(left) is c_ast.Constant and type(right) is c_ast.ID:
                return Value(variable=right.name, constante=int(left.value))
            elif type(left) is c_ast.ID and type(right) is c_ast.Constant:
                return Value(variable=left.name, constante=int(right.value))
            else:
                print(ast_node)
                if op is "+":
                    return Value(constante=(int(left.value)+int(right.value)))
                elif op is "-":
                    return Value(constante=(int(left.value)-int(right.value)))          
        
    def __str__(self):
        if self.variable is None:
            return str(self.constante)
        return self.variable + ' + ' + str(self.constante)

In [21]:
print(Value(-10, 'a'))

a + -10


In [22]:
class Variable():
    """
        A variable is an interval or a singleton
    """
    def __init__(self):
        pass

class Singleton(Variable):
    """
        Représente un singleton
        
        Attributs
        ----------
        value: @Value
            valeur du singleton
    """
    def __init__(self, value):
        super()
        self.value = value
    
    def __str__(self):
        return ("{" + str(self.value) + "}")
    
class Interval(Variable):
    """
        Représente un interval
        
        Attributs
        ----------
        min: @Value
            Minimum de l'intervalle
        max: @Value 
            Maximum de l'intervalle
    """
    def __init__(self, minimum, maximum):
        self.min = minimum
        self.max = maximum
    
    def __str__(self):
        return ("[" + str(self.min) + "," + str(self.max) + "]")

In [23]:
a = Interval(Value(constante=-inf), Value(constante=inf))
b = Singleton(Value(constante=0))

print(a, b)

[-inf,inf] {0}


In [24]:
class Condition:
    """
        Represents a condition
        
        Arguments
        ----------
        coord: String
            lines of the condition
        left: @Variable
            left operand
        operator: String
            operator
        right: @Variable
            right operand
    """
    def __init__(self, condition = None, condition_boolean = True):
        self.condition = condition
        (self.left, self.operator, self.right) = self.handleCondition()
    
    def handleCondition(self):
        left_operand = Singleton(Value.compute(self.condition.left))
        right_operand = Singleton(Value.compute(self.condition.right))
        operator = self.condition.op
        
        if operator == ">":
            return (right_operand, "<", left_operand)
        elif operator == ">=":
            return (right_operand, "<=", left_operand)
        elif operator == "%":
            raise Exception("Modulo is not defined")
        else:
            return (left_operand, operator, right_operand)
    
    def reverse(condition):
        reversed_condition = copy(condition)
        reversed_condition.condition_boolean = !condition.condition_boolean
        if condition.operator == "<":
            reversed_condition.left, reversed_condition.operator, reversed_condition.right = \
                condition.right, "<=", condition.left
        elif condition.operator == "<=":
            reversed_condition.left, reversed_condition.operator, reversed_condition.right = \
                condition.right, "<", condition.left
        elif condition.operator == "==":
            reversed_condition.operator = "!="
        elif condition.operator == "!=":
            reversed_condition.operator = "=="
        else:
            raise Exception("Wrong operator for conditions: " + condition.operator)
        return reversed_condition
    
    def __str__(self):
        return str(self.left) + " " + self.operator + " " + str(self.right)

In [25]:
if_condition = foo_graph.body.block_items[3].cond
condition = Condition(if_condition)
print(condition)

{a + 4} != {a + 4}


In [26]:
class Context:
    """
        Context of execution of the program
        
        Arguments
        ----------
        conditions: list[@Condition]
            conditions evaluated by the context
        variables: list[@Variable]
            variables in the context
        next_nodes: list[@c_ast.Node]
            next nodes to visit
        state: boolean
            true if the context executes, false if it fails
    """
    def __init__(self, condition = None, next_nodes = [], previous_context = None):
        if previous_context is not None:
            self.conditions = previous_context.conditions + [condition]
            self.variables = previous_context.variables
            self.next_nodes = previous_context.next_nodes + next_nodes
            self.checkCondition(condition)
            
        else:
            self.conditions = []
            self.variables = {}
            self.next_nodes = next_nodes
    
    def addParam(self, param):
        name = param.name
        types = param.type.type.names
        if 'int' not in types:
            raise Exception('Only int type allowed')
        if 'unsigned' in types:
            self.variables[name] = Interval(Value(constante = 0), Value(constante = inf))
        else:
            self.variables[name] = Interval(Value(constante = -inf), Value(constante = inf))
    
    def addVariable(self, node_decl):
        name = node_decl.name
        types = node_decl.type.type.names
        if 'int' not in types:
            raise Exception('Only int type allowed')
        if node_decl.init is not None:
            variable = Singleton(Value.compute(node_decl.init))
        else:
            variable = Singleton(Value(constante=0))
        self.variables[name] = variable
    
    def assign(self, node_assign):
        name = node_assign.lvalue.name
        variable = Singleton(Value.compute(node_assign.rvalue))
        self.variables[name] = variable
    
    def pop(self):
        if self.next_nodes != []:
            return self.next_nodes.pop(0)
        return None
        
    def checkCondition(self, condition):
        """
        Si self.state = True condition possible
        Si self.state = False condition impossible donc le branchement est inutile
        """
        self.state = True
        if condition.left.value.variable is None and condition.right.value.variable is None:
            self.state = Operation.compareConstante(condition.left.value.constante,\
                                                    condition.right.value.constante,\
                                                    condition.operator)
        elif condition.left.value.variable is not None and\
             condition.right.value.variable is not None:
            if condition.left.value.variable == condition.left.value.variable:
                 self.state = Operation.compareConstante(condition.left.value.constante,\
                                                    condition.right.value.constante,\
                                                    condition.operator)
            else:
                pass
                    
        # simplification des deux listes représentant les variables
        # propagation des singletons grâce à leur valeur trouvée dans la bibliothèque
        # si il y a encore des variables entièrement non définies de chaque côté -> 
        # deux contexte et pas d'update
        # si non définie d'un seul côté -> on a les deux contextes et update de la valeur 
        # de la variable
        # sinon -> un seul contexte et pas d'update
        return
    
    def __str__(self):
        return "Context: [conditions: " + \
               ", ".join(str(condition) for condition in self.conditions) + \
               "]; [variables: " + ", ".join(variable + ":" + str(self.variables[variable]) \
                                          for variable in self.variables) + "]"

In [27]:
class Operation():
    
    def compareConstante(left, right, operator):
        if operator == "<" and left >= right:
            return False
        elif operator == "<=" and left > right:
            return False
        elif operator == "==" and left != right:
            return False
        elif operator == "!=" and left == right:
            return False
        return True
    
    def computeVariable(variable):
        pass

In [28]:
def isIf(node):
    return type(node) == c_ast.If

def isDeclaration(node):
    return type(node) == c_ast.Decl

def isAssignment(node):
    return type(node) == c_ast.Assignment

def getCoord(node):
    return node.coord

In [29]:
def visitParamsFunction(context, params):
    for param in params:
        context.addParam(param)

In [30]:
def visitNode(context):
    node = context.pop()
    if node is not None:
        if isIf(node):
            condition = Condition(node.cond)
            context1 = Context(condition = condition, next_nodes = [node.iftrue],\
                               previous_context = context)
            context2 = Context(condition = Condition.reverse(condition),\
                               next_nodes = [node.iffalse], previous_context = context)
            if context1.state and context2.state:
                return visitNode(context1) + visitNode(context2)
            elif context1.state and not context2.state:
                return visitNode(context1) + [context2]
            elif not context1.state and context2.state:
                return visitNode(context2) + [context1]
            else:
                return [context1, context2]
        elif isDeclaration(node):
            context.addVariable(node)
        elif isAssignment(node):
            context.assign(node)
        else:
            for (string, child) in node.children():
                context.next_nodes.append(child)
        return visitNode(context)
    return []

In [31]:
def visitFunction(function):
    context = Context(next_nodes = [function.body])
    visitParamsFunction(context, function.decl.type.args.params)
    return visitNode(context)

In [32]:
contexts = visitFunction(foo_graph)

for context in contexts:
    print("La condition: " + str(context.conditions[-1]) + " n'est pas possible à la ligne " +\
          str(context.conditions[-1].condition.coord))

La condition: {a + 4} != {a + 4} n'est pas possible à la ligne ./C files/if.c:18:9
